<a href="https://colab.research.google.com/github/fef40/bsm_greeks/blob/main/MarteauD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import math
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

#Paramètres de marché
while True :
  spot=float(input("Spot du sous-jacent ?"))
  if spot>0:
    break
  else:
    print("Le spot ne peut être négatif bon sang")
while True :
  sigma=float(input("Sigma du sous-jacent ?"))
  if sigma>0:
    break
  else:
    print("Une vol négative ? Et puis quoi encore ?")
r=float(input("Quel est le taux sans risque ?"))

#Ask for the option trading strategy

calls=[]

nombre_de_calls=int(input("Nombre de calls?"))
if nombre_de_calls>0:
  for i in range(nombre_de_calls):
    while True :
      K=float(input(f'Strike du call {i+1} ?'))
      if K>0:
        break
      else:
        print("Erreur: le strike ne peut être négatif")
    while True :
      T=float(input(f'Matu du call {i+1} ?'))
      if T>0:
        break
      else:
        print("Erreur: la maturité ne peut être négative")
    while True :
      quantite=float(input(f'Quantité d options {i+1} ?'))
      if quantite>=0:
        break
      else:
        print("Erreur: la quantité ne peut être négative")
    while True:
      position=input('Position (long/short)?').strip().lower()
      if position in ["long","short"]:
        break
      else:
        print("Erreur : il faut taper long ou short")
    calls.append({"strike":K,"matu":T,"position":position,"quantite":quantite,"type":"call"})
else:
      print("Pas de calls")
print("Calls:",calls)


puts=[]

nombre_de_puts=int(input("Nombre de puts?"))
if nombre_de_puts>0:
  for i in range(nombre_de_puts):
    while True :
      K=float(input(f'Strike du put {i+1} ?'))
      if K>0:
        break
      else:
        print("Erreur: le strike ne peut être négatif")
    while True :
      T=float(input(f'Matu du put {i+1} ?'))
      if T>0:
        break
      else:
        print("Erreur: la maturité ne peut être négative")
    while True :
      quantite=float(input(f'Quantité d options {i+1} ?'))
      if quantite>=0:
        break
      else:
        print("Erreur: la quantité ne peut être négative")
    while True:
      position=input('Position (long/short)?').strip().lower()
      if position in ["long","short"]:
        break
      else:
        print("Erreur : il faut taper long ou short")
    puts.append({"strike":K,"matu":T,"position":position,"quantite":quantite,"type":"put"})
else:
      print("Pas de put")
print("Puts:",puts)

position_delta_1=float(input("Quelle est la position delta one en notionnel absolu ?"))
if position_delta_1>0:
  position_delta1=input('Position (long/short)?').strip().lower()
  print("On est",position_delta1,position_delta_1,"en delta one")
else:
  print("Pas de position delta One")

#Calcul des grecques

def calcul_des_grecques(leg,spot,r,sigma):
  K=leg["strike"]
  T=leg["matu"]
  q=leg["quantite"]
  position=leg["position"]
  type_doption=leg["type"]
  sign=+1 if position=="long" else -1
  d1=(math.log(spot/K)+(r+0.5*sigma**2)*T)/(sigma*math.sqrt(T))
  d2=d1-sigma*math.sqrt(T)
  if type_doption=="call":
      delta=norm.cdf(d1)*sign*q
  else:
      delta=(norm.cdf(d1)-1)*sign*q
  gamma=((norm.pdf(d1))/(spot*sigma*math.sqrt(T)))*sign
  vega=(spot*norm.pdf(d1)*math.sqrt(T))*sign
  return delta, gamma, vega


print("="*30)

delta_total_calls=0
gamma_total_calls=0
vega_total_calls=0

for leg in calls:
    delta, gamma, vega= calcul_des_grecques(leg, spot, r, sigma)
    delta_total_calls+=delta
    gamma_total_calls+=gamma
    vega_total_calls+=vega
    leg["delta"]=delta
    leg["gamma"]=gamma
    leg["vega"]=vega
print("Le delta cumulé des calls est de",delta_total_calls)
print("Le gamma cumulé des calls est de",gamma_total_calls)
print("Le vega cumulé des calls est de",vega_total_calls)

print("="*30)

delta_total_puts=0
gamma_total_puts=0
vega_total_puts=0

for leg in puts :
    delta, gamma, vega= calcul_des_grecques(leg, spot, r, sigma)
    delta_total_puts+= delta
    gamma_total_puts+=gamma
    vega_total_puts+=vega
    leg["delta"]=delta
    leg["gamma"]=gamma
    leg["vega"]=vega
print("Le delta cumulé des puts est de",delta_total_puts)
print("Le gamma cumulé des puts est de",gamma_total_puts)
print("Le vega cumulé des puts est de",vega_total_puts)

contribution_delta_one=0

if position_delta_1==0 :
  contribution_delta_one = 0
else:
    deltaone=(position_delta_1/spot)
    if position_delta1=="long":
        sign_delta_one= +1
    else:
        sign_delta_one = -1
    contribution_delta_one= deltaone*sign_delta_one

delta_total = delta_total_calls + delta_total_puts + contribution_delta_one
gamma_total = gamma_total_calls + gamma_total_puts
vega_total = vega_total_calls + vega_total_puts
print("="*30)
print("Le delta de la stratégie est de",delta_total)
print("Le gamma de la stratégie est de",gamma_total)
print("Le vega de la stratégie est de",vega_total)
print("="*30)

df=pd.DataFrame({
    "Position":["Calls","Puts","Delta One","Total"],
    "Delta": [delta_total_calls, delta_total_puts, contribution_delta_one, delta_total],
    "Gamma": [gamma_total_calls, gamma_total_puts,0, delta_total],
    "Vega": [vega_total_calls, vega_total_puts,0, vega_total]
})

print(df)


Spot du sous-jacent ?100
Sigma du sous-jacent ?0.2
Quel est le taux sans risque ?0.05
Nombre de calls?1
Strike du call 1 ?100
Matu du call 1 ?0.01
Quantité d options 1 ?1
Position (long/short)?0
Erreur : il faut taper long ou short
Position (long/short)?long
Calls: [{'strike': 100.0, 'matu': 0.01, 'position': 'long', 'quantite': 1.0, 'type': 'call'}]
Nombre de puts?0
Pas de put
Puts: []
Quelle est la position delta one en notionnel absolu ?0
Pas de position delta One
Le delta cumulé des calls est de 0.5139601295627583
Le gamma cumulé des calls est de 0.19934900153612786
Le vega cumulé des calls est de 3.986980030722558
Le delta cumulé des puts est de 0
Le gamma cumulé des puts est de 0
Le vega cumulé des puts est de 0
Le delta de la stratégie est de 0.5139601295627583
Le gamma de la stratégie est de 0.19934900153612786
Le vega de la stratégie est de 3.986980030722558
    Position    Delta     Gamma     Vega
0      Calls  0.51396  0.199349  3.98698
1       Puts  0.00000  0.000000  0.000